# PyThaiNLP Get Started

Code examples for basic functions in PyThaiNLP https://github.com/PyThaiNLP/pythainlp

In [19]:
# # pip install required modules
# # uncomment if running from colab
# # see list of modules in `requirements` and `extras`
# # in https://github.com/PyThaiNLP/pythainlp/blob/dev/setup.py

!pip install -q python-crfsuite
!pip install -q torch
!pip install -q pythainlp
!pip install -q epitran

## Import PyThaiNLP

In [20]:
import pythainlp

pythainlp.__version__

'5.2.0'

## Thai Characters

PyThaiNLP provides some ready-to-use Thai character set (e.g. Thai consonants, vowels, tonemarks, symbols) as a string for convenience. There are also few utility functions to test if a string is in Thai or not.

In [21]:
pythainlp.thai_characters

'กขฃคฅฆงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรลวศษสหฬอฮฤฦะัาำิีึืุูเแโใไๅํ็่้๊๋ฯฺๆ์ํ๎๏๚๛๐๑๒๓๔๕๖๗๘๙฿'

In [22]:
len(pythainlp.thai_characters)

88

In [23]:
pythainlp.thai_consonants

'กขฃคฅฆงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรลวศษสหฬอฮ'

In [24]:
len(pythainlp.thai_consonants)

44

In [25]:
"๔" in pythainlp.thai_digits  # check if Thai digit "4" is in the character set

True

## Checking if a string contains Thai character or not, or how many

In [26]:
import pythainlp.util

pythainlp.util.isthai("ก")

True

In [27]:
pythainlp.util.isthai("(ก.พ.)")

False

In [28]:
pythainlp.util.isthai("(ก.พ.)", ignore_chars=".()")

True

`counthai()` returns proportion of Thai characters in the text. It will ignore non-alphabets by default.

In [29]:
pythainlp.util.countthai("วันอาทิตย์ที่ 24 มีนาคม 2562")

100.0

You can specify characters to be ignored, using `ignore_chars=` parameter.

In [30]:
pythainlp.util.countthai("วันอาทิตย์ที่ 24 มีนาคม 2562", ignore_chars="")

67.85714285714286

## Collation

Sorting according to Thai dictionary.

In [31]:
from pythainlp.util import collate

thai_words = ["ค้อน", "กระดาษ", "กรรไกร", "ไข่", "ผ้าไหม"]
collate(thai_words)

['กรรไกร', 'กระดาษ', 'ไข่', 'ค้อน', 'ผ้าไหม']

In [32]:
collate(thai_words, reverse=True)

['ผ้าไหม', 'ค้อน', 'ไข่', 'กระดาษ', 'กรรไกร']

## Date/Time Format and Spellout

### Date/Time Format

Get Thai day and month names with Thai Buddhist Era (B.E.).
Use [formatting directives](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes) similar to `datetime.strftime()`.

In [33]:
import datetime
from pythainlp.util import thai_strftime

fmt = "%Aที่ %-d %B พ.ศ. %Y เวลา %H:%M น. (%a %d-%b-%y)"
date = datetime.datetime(1976, 10, 6, 1, 40)

thai_strftime(date, fmt)

'วันพุธที่ 6 ตุลาคม พ.ศ. 2519 เวลา 01:40 น. (พ 06-ต.ค.-19)'

From version 2.2, these modifiers can be applied right before the main directive:

- \- *(minus)* Do not pad a numeric result string (also available in version 2.1)
- _ *(underscore)* Pad a numeric result string with spaces
- 0 *(zero)* Pad a number result string with zeros
- ^ Convert alphabetic characters in result string to upper case
- \# Swap the case of the result string
- O *(letter o)* Use the locale's alternative numeric symbols (Thai digit)

In [34]:
thai_strftime(date, "%d %b %y")

'06 ต.ค. 19'

In [35]:
thai_strftime(date, "%d %b %Y")

'06 ต.ค. 2519'

### Time Spellout

*Note: `thai_time()` will be renamed to `time_to_thaiword()` in version 2.2.*

In [36]:
from pythainlp.util import thai_time

thai_time("00:14:29")

ImportError: cannot import name 'thai_time' from 'pythainlp.util' (/usr/local/lib/python3.12/dist-packages/pythainlp/util/__init__.py)

The way to spellout can be chosen, using `fmt` parameter.
It can be `24h`, `6h`, or `m6h`. Try one by yourself.

In [ ]:
thai_time("00:14:29", fmt="6h")

Precision of spellout can be chosen as well. Using `precision` parameter.
It can be `m` for minute-level, `s` for second-level, or `None` for only read the non-zero value.

In [ ]:
thai_time("00:14:29", precision="m")

In [ ]:
print(thai_time("8:17:00", fmt="6h"))
print(thai_time("8:17:00", fmt="m6h", precision="s"))
print(thai_time("18:30:01", fmt="m6h", precision="m"))
print(thai_time("13:30:01", fmt="6h", precision="m"))

We can also pass `datetime` and `time` objects to `thai_time()`.

In [ ]:
import datetime

time = datetime.time(13, 14, 15)
thai_time(time)

In [ ]:
time = datetime.datetime(10, 11, 12, 13, 14, 15)
thai_time(time, fmt="6h", precision="m")

## Tokenization and Segmentation

At sentence, word, and sub-word levels.

### Sentence

Default sentence tokenizer is "crfcut". Tokenization engine can be chosen ussing `engine=` parameter.

In [37]:
from pythainlp import sent_tokenize

text = ("พระราชบัญญัติธรรมนูญการปกครองแผ่นดินสยามชั่วคราว พุทธศักราช ๒๔๗๕ "
        "เป็นรัฐธรรมนูญฉบับชั่วคราว ซึ่งถือว่าเป็นรัฐธรรมนูญฉบับแรกแห่งราชอาณาจักรสยาม "
        "ประกาศใช้เมื่อวันที่ 27 มิถุนายน พ.ศ. 2475 "
        "โดยเป็นผลพวงหลังการปฏิวัติเมื่อวันที่ 24 มิถุนายน พ.ศ. 2475 โดยคณะราษฎร")

print("default (crfcut):")
print(sent_tokenize(text))
print("\nwhitespace+newline:")
print(sent_tokenize(text, engine="whitespace+newline"))

default (crfcut):
['พระราชบัญญัติธรรมนูญการปกครองแผ่นดินสยามชั่วคราว พุทธศักราช ๒๔๗๕ เป็นรัฐธรรมนูญฉบับชั่วคราว ', 'ซึ่งถือว่าเป็นรัฐธรรมนูญฉบับแรกแห่งราชอาณาจักรสยาม ', 'ประกาศใช้เมื่อวันที่ 27 มิถุนายน พ.ศ.', ' 2475 โดยเป็นผลพวงหลังการปฏิวัติเมื่อวันที่ 24 มิถุนายน พ.ศ.', ' 2475 โดยคณะราษฎร']

whitespace+newline:
['พระราชบัญญัติธรรมนูญการปกครองแผ่นดินสยามชั่วคราว', 'พุทธศักราช', '๒๔๗๕', 'เป็นรัฐธรรมนูญฉบับชั่วคราว', 'ซึ่งถือว่าเป็นรัฐธรรมนูญฉบับแรกแห่งราชอาณาจักรสยาม', 'ประกาศใช้เมื่อวันที่', '27', 'มิถุนายน', 'พ.ศ.', '2475', 'โดยเป็นผลพวงหลังการปฏิวัติเมื่อวันที่', '24', 'มิถุนายน', 'พ.ศ.', '2475', 'โดยคณะราษฎร']


### Word
Default word tokenizer ("newmm") use maximum matching algorithm.

In [38]:
from pythainlp import word_tokenize

text = "ก็จะรู้ความชั่วร้ายที่ทำไว้     และคงจะไม่ยอมให้ทำนาบนหลังคน "

print("default (newmm):")
print(word_tokenize(text))
print("\nnewmm and keep_whitespace=False:")
print(word_tokenize(text, keep_whitespace=False))

default (newmm):
['ก็', 'จะ', 'รู้ความ', 'ชั่วร้าย', 'ที่', 'ทำ', 'ไว้', '     ', 'และ', 'คงจะ', 'ไม่', 'ยอมให้', 'ทำนาบนหลังคน', ' ']

newmm and keep_whitespace=False:
['ก็', 'จะ', 'รู้ความ', 'ชั่วร้าย', 'ที่', 'ทำ', 'ไว้', 'และ', 'คงจะ', 'ไม่', 'ยอมให้', 'ทำนาบนหลังคน']


Other algorithm can be chosen. We can also create a tokenizer with a custom dictionary.

In [39]:
from pythainlp import word_tokenize, Tokenizer

text = "กฎหมายแรงงานฉบับปรับปรุงใหม่ประกาศใช้แล้ว"

print("newmm  :", word_tokenize(text))  # default engine is "newmm"
print("longest:", word_tokenize(text, engine="longest"))

words = ["แรงงาน"]
custom_tokenizer = Tokenizer(words)
print("newmm (custom dictionary):", custom_tokenizer.word_tokenize(text))

newmm  : ['กฎหมายแรงงาน', 'ฉบับ', 'ปรับปรุง', 'ใหม่', 'ประกาศ', 'ใช้แล้ว']
longest: ['กฎหมายแรงงาน', 'ฉบับ', 'ปรับปรุง', 'ใหม่', 'ประกาศใช้', 'แล้ว']
newmm (custom dictionary): ['กฎหมาย', 'แรงงาน', 'ฉบับปรับปรุงใหม่ประกาศใช้แล้ว']


Default word tokenizer use a word list from `pythainlp.corpus.common.thai_words()`.
We can get that list, add/remove words, and create new tokenizer from the modified list.

In [40]:
from pythainlp.corpus.common import thai_words
from pythainlp import Tokenizer

text = "นิยายวิทยาศาสตร์ของไอแซค อสิมอฟ"

print("default dictionary:", word_tokenize(text))

words = set(thai_words())  # thai_words() returns frozenset
words.add("ไอแซค")  # Isaac
words.add("อสิมอฟ")  # Asimov
custom_tokenizer = Tokenizer(words)
print("custom dictionary :", custom_tokenizer.word_tokenize(text))

default dictionary: ['นิยาย', 'วิทยาศาสตร์', 'ของ', 'ไอแซค', ' ', 'อสิ', 'มอ', 'ฟ']
custom dictionary : ['นิยาย', 'วิทยาศาสตร์', 'ของ', 'ไอแซค', ' ', 'อสิมอฟ']


We can also, alternatively, create a dictionary trie, using `pythainlp.util.Trie()` function, and pass it to a default tokenizer.

In [41]:
from pythainlp.corpus.common import thai_words
from pythainlp.util import Trie

text = "ILO87 ว่าด้วยเสรีภาพในการสมาคมและการคุ้มครองสิทธิในการรวมตัว ILO98 ว่าด้วยสิทธิในการรวมตัวและการร่วมเจรจาต่อรอง"

print("default dictionary:", word_tokenize(text))

new_words = {"ILO87", "ILO98", "การร่วมเจรจาต่อรอง", "สิทธิในการรวมตัว", "เสรีภาพในการสมาคม", "แรงงานสัมพันธ์"}
words = new_words.union(thai_words())

custom_dictionary_trie = Trie(words)
print("custom dictionary :", word_tokenize(text, custom_dict=custom_dictionary_trie))

default dictionary: ['ILO', '87', ' ', 'ว่าด้วย', 'เสรีภาพ', 'ใน', 'การสมาคม', 'และ', 'การ', 'คุ้มครอง', 'สิทธิ', 'ใน', 'การ', 'รวมตัว', ' ', 'ILO', '98', ' ', 'ว่าด้วย', 'สิทธิ', 'ใน', 'การ', 'รวมตัว', 'และ', 'การ', 'ร่วม', 'เจรจา', 'ต่อรอง']
custom dictionary : ['ILO87', ' ', 'ว่าด้วย', 'เสรีภาพในการสมาคม', 'และ', 'การ', 'คุ้มครอง', 'สิทธิในการรวมตัว', ' ', 'ILO98', ' ', 'ว่าด้วย', 'สิทธิในการรวมตัว', 'และ', 'การร่วมเจรจาต่อรอง']


Testing different tokenization engines

In [42]:
speedtest_text = """
ครบรอบ 14 ปี ตากใบ เช้าวันนั้น 25 ต.ค. 2547 ผู้ชุมนุมชายกว่า 1,370 คน
ถูกโยนขึ้นรถยีเอ็มซี 22 หรือ 24 คัน นอนซ้อนกันคันละ 4-5 ชั้น เดินทางจากสถานีตำรวจตากใบ ไปไกล 150 กิโลเมตร
ไปถึงค่ายอิงคยุทธบริหาร ใช้เวลากว่า 6 ชั่วโมง / ในอีกคดีที่ญาติฟ้องร้องรัฐ คดีจบลงที่การประนีประนอมยอมความ
กระทรวงกลาโหมจ่ายค่าสินไหมทดแทนรวม 42 ล้านบาทให้กับญาติผู้เสียหาย 79 ราย
ปิดหีบและนับคะแนนเสร็จแล้ว ที่หน่วยเลือกตั้งที่ 32 เขต 13 แขวงหัวหมาก เขตบางกะปิ กรุงเทพมหานคร
ผู้สมัคร ส.ส. และตัวแทนพรรคการเมืองจากหลายพรรคต่างมาเฝ้าสังเกตการนับคะแนนอย่างใกล้ชิด โดย
ฐิติภัสร์ โชติเดชาชัยนันต์ จากพรรคพลังประชารัฐ และพริษฐ์ วัชรสินธุ จากพรรคประชาธิปัตย์ได้คะแนน
96 คะแนนเท่ากัน
เช้าวันอาทิตย์ที่ 21 เมษายน 2019 ซึ่งเป็นวันอีสเตอร์ วันสำคัญของชาวคริสต์
เกิดเหตุระเบิดต่อเนื่องในโบสถ์คริสต์และโรงแรมอย่างน้อย 7 แห่งในประเทศศรีลังกา
มีผู้เสียชีวิตแล้วอย่างน้อย 156 คน และบาดเจ็บหลายร้อยคน ยังไม่มีข้อมูลว่าผู้ก่อเหตุมาจากฝ่ายใด
จีนกำหนดจัดการประชุมข้อริเริ่มสายแถบและเส้นทางในช่วงปลายสัปดาห์นี้ ปักกิ่งยืนยันว่า
อภิมหาโครงการเชื่อมโลกของจีนไม่ใช่เครื่องมือแผ่อิทธิพล แต่ยินดีรับฟังข้อวิจารณ์ เช่น ประเด็นกับดักหนี้สิน
และความไม่โปร่งใส รัฐบาลปักกิ่งบอกว่า เวทีประชุม Belt and Road Forum ในช่วงวันที่ 25-27 เมษายน
ถือเป็นงานการทูตที่สำคัญที่สุดของจีนในปี 2019
"""

In [43]:
# Speed test: Calling "longest" engine through word_tokenize wrapper
%time tokens = word_tokenize(speedtest_text, engine="longest")

CPU times: user 193 ms, sys: 884 µs, total: 194 ms
Wall time: 194 ms


In [44]:
# Speed test: Calling "newmm" engine through word_tokenize wrapper
%time tokens = word_tokenize(speedtest_text, engine="newmm")

CPU times: user 3.02 ms, sys: 1 ms, total: 4.03 ms
Wall time: 3.99 ms


In [45]:
# Speed test: Calling "newmm" engine through word_tokenize wrapper
%time tokens = word_tokenize(speedtest_text, engine="newmm-safe")

CPU times: user 6.17 ms, sys: 19 µs, total: 6.19 ms
Wall time: 6.2 ms


In [ ]:
#!pip install attacut
# Speed test: Calling "attacut" engine through word_tokenize wrapper
%time tokens = word_tokenize(speedtest_text, engine="attacut")

Get all possible segmentations

In [ ]:
from pythainlp.tokenize.multi_cut import find_all_segment, mmcut, segment

find_all_segment("มีความเป็นไปได้อย่างไรบ้าง")

### Subword, syllable, and Thai Character Cluster (TCC)

Tokenization can also be done at subword level, either syllable or Thai Character Cluster (TCC).

- Syllable segmentation is using [`ssg`](https://github.com/ponrawee/ssg), a CRF syllable segmenter for Thai by Ponrawee Prasertsom.
- TCC is smaller than syllable. For information about TCC, see [Character Cluster Based Thai Information Retrieval](https://www.researchgate.net/publication/2853284_Character_Cluster_Based_Thai_Information_Retrieval) (Theeramunkong et al. 2004).

#### Subword tokenization
Default subword tokenization engine is `tcc`, which will use Thai Character Cluster (TCC) as a subword unit.

In [46]:
from pythainlp import subword_tokenize

subword_tokenize("ประเทศไทย")  # default subword unit is TCC

['ป', 'ระ', 'เท', 'ศ', 'ไท', 'ย']

#### Syllable tokenization
Default syllable tokenization engine is `dict`, which will use `newmm` word tokenization engine with a custom dictionary contains known syllables in Thai language.

In [ ]:
from pythainlp.tokenize import syllable_tokenize

text = "อับดุลเลาะ อีซอมูซอ สมองบวมรุนแรง"

syllable_tokenize(text)  # default engine is "dict"

External [`ssg`](https://github.com/ponrawee/ssg) engine call be called. Note that `ssg` engine ommitted whitespaces in the output tokens.

In [ ]:
syllable_tokenize(text, engine="ssg")  # use "ssg" for syllable

#### Low-level subword operations

These low-level TCC operations can be useful for some pre-processing tasks. Like checking if it's ok to cut a string at a certain point or to find typos.

In [ ]:
from pythainlp.tokenize import tcc

tcc.segment("ประเทศไทย")

In [ ]:
tcc.tcc_pos("ประเทศไทย")  # return positions

In [ ]:
for ch in tcc.tcc("ประเทศไทย"):  # TCC generator
    print(ch, end='-')

## Transliteration

There are two types of transliteration here: romanization and transliteration.

- Romanization will render Thai words in the Latin alphabet using the [Royal Thai General System of Transcription (RTGS)](https://en.wikipedia.org/wiki/Royal_Thai_General_System_of_Transcription).
  - Two engines are supported here: a simple `royin` engine (default) and a more accurate `thai2rom` engine.
- Transliteration here, in PyThaiNLP context, means the sound representation of a string.
  - Two engines are supported here: `ipa` (International Phonetic Alphabet system, using [Epitran](https://github.com/dmort27/epitran)) (default) and `icu` (International Components for Unicode, using [PyICU](https://github.com/ovalhub/pyicu)).

In [ ]:
from pythainlp.transliterate import romanize

romanize("แมว")  # output: 'maeo'

In [ ]:
romanize("ภาพยนตร์")  # output: 'phapn' (*obviously wrong)

In [ ]:
from pythainlp.transliterate import transliterate

transliterate("แมว")  # output: 'mɛːw'

In [ ]:
transliterate("ภาพยนตร์")  # output: 'pʰaːpjanot'

## Normalization

`normalize()` removes zero-width spaces (ZWSP and ZWNJ), duplicated spaces, repeating vowels, and dangling characters. It also reorder vowels and tone marks during the process of removing repeating vowels.

In [ ]:
from pythainlp.util import normalize

normalize("เเปลก") == "แปลก"  # เ เ ป ล ก  vs แ ป ล ก

The string below contains a non-standard order of Thai characters,
Sara Aa (following vowel) + Mai Ek (upper tone mark).
`normalize()` will reorder it to Mai Ek + Sara Aa.

In [ ]:
text = "เกา่"
normalize(text)

This can be useful for string matching, including tokenization.

In [ ]:
from pythainlp import word_tokenize

text = "เก็บวันน้ี พรุ่งน้ีก็เกา่"

print("tokenize immediately:")
print(word_tokenize(text))
print("\nnormalize, then tokenize:")
print(word_tokenize(normalize(text)))

The string below contains repeating vowels (multiple Sara A in a row)
normalize() will keep only one of them. It can be use to reduce variations in spellings, useful for classification task.

In [ ]:
normalize("เกะะะ")

Internally, `normalize()` is just a series of function calls like this:
```
text = remove_zw(text)
text = remove_dup_spaces(text)
text = remove_repeat_vowels(text)
text = remove_dangling(text)
```

If you don't like the behavior of default `normalize()`, you can call those functions shown above, also `remove_tonemark()` and `reorder_vowels()`, individually from `pythainlp.util`, to customize your own normalization.

## Digit conversion

Thai text sometimes use Thai digits. This can reduce performance for classification and searching. PyThaiNP provides few utility functions to deal with this.

In [ ]:
from pythainlp.util import arabic_digit_to_thai_digit, thai_digit_to_arabic_digit, digit_to_text

text = "ฉุกเฉินที่ยุโรปเรียก 112 ๑๑๒"

arabic_digit_to_thai_digit(text)

In [ ]:
thai_digit_to_arabic_digit(text)

In [ ]:
digit_to_text(text)

## Soundex

"Soundex is a phonetic algorithm for indexing names by sound." ([Wikipedia](https://en.wikipedia.org/wiki/Soundex)). PyThaiNLP provides three kinds of Thai soundex.

In [ ]:
from pythainlp.soundex import lk82, metasound, udom83

# check equivalence
print(lk82("รถ") == lk82("รด"))
print(udom83("วรร") == udom83("วัน"))
print(metasound("นพ") == metasound("นภ"))

In [ ]:
texts = ["บูรณะ", "บูรณการ", "มัก", "มัค", "มรรค", "ลัก", "รัก", "รักษ์", ""]
for text in texts:
    print(
        "{} - lk82: {} - udom83: {} - metasound: {}".format(
            text, lk82(text), udom83(text), metasound(text)
        )
    )

## Spellchecking

Default spellchecker uses [Peter Norvig's algorithm](http://www.norvig.com/spell-correct.html) together with word frequency from Thai National Corpus (TNC).

`spell()` returns a list of all possible spellings.

In [ ]:
from pythainlp import spell

spell("เหลืยม")

`correct()` returns the most likely spelling.

In [ ]:
from pythainlp import correct

correct("เหลืยม")

## Spellchecking - Custom dictionary and word frequency

Custom dictionary can be provided when creating spellchecker.

When create a `NorvigSpellChecker` object, you can pass a custom dictionary to `custom_dict` parameter.

`custom_dict` can be:
- a dictionary (`dict`), with words (`str`) as keys and frequencies (`int`) as values; or
- a list, a tuple, or a set of (word, frequency) tuples; or
- a list, a tuple, or a set of just words, without their frequencies -- in this case `1` will be assigned to every words.

In [ ]:
from pythainlp.spell import NorvigSpellChecker

user_dict = [("เหลียม", 50), ("เหลือม", 1000), ("เหลียว", 1000000)]
checker = NorvigSpellChecker(custom_dict=user_dict)

checker.spell("เหลืยม")


As you can see, our version of `NorvigSpellChecker` gives the edit distance a priority over a word frequency.

You can use word frequencies from Thai National Corpus and Thai Textbook Corpus as well.

By default, `NorvigSpellChecker` uses Thai National Corpus.

In [ ]:
from pythainlp.corpus import ttc  # Thai Textbook Corpus

checker = NorvigSpellChecker(custom_dict=ttc.word_freqs())

checker.spell("เหลืยม")

In [ ]:
checker.correct("เหลืยม")

To check the current dictionary of a spellchecker:

In [ ]:
list(checker.dictionary())[1:10]

We can also apply conditions and filter function to dictionary when creating spellchecker.

In [ ]:
checker = NorvigSpellChecker()  # use default filter (remove any word with number or non-Thai character)
len(checker.dictionary())

In [ ]:
checker = NorvigSpellChecker(min_freq=5, min_len=2, max_len=15)
len(checker.dictionary())

In [ ]:
checker_no_filter = NorvigSpellChecker(dict_filter=None)  # use no filter
len(checker_no_filter.dictionary())

In [ ]:
def remove_yamok(word):
    return False if "ๆ" in word else True

checker_custom_filter = NorvigSpellChecker(dict_filter=remove_yamok)  # use custom filter
len(checker_custom_filter.dictionary())

## Part-of-Speech Tagging

In [ ]:
from pythainlp.tag import pos_tag, pos_tag_sents

pos_tag(["การ","เดินทาง"])

In [ ]:
sents = [["ประกาศสำนักนายกฯ", " ", "ให้",
    " ", "'พล.ท.สรรเสริญ แก้วกำเนิด'", " ", "พ้นจากตำแหน่ง",
    " ", "ผู้ทรงคุณวุฒิพิเศษ", "กองทัพบก", " ", "กระทรวงกลาโหม"],
    ["และ", "แต่งตั้ง", "ให้", "เป็น", "'อธิบดีกรมประชาสัมพันธ์'"]]

pos_tag_sents(sents)

## Named-Entity Tagging

The tagger use BIO scheme:
- B - beginning of entity
- I - inside entity
- O - outside entity

In [47]:
#!pip3 install pythainlp[ner]
from pythainlp.tag.thainer import ThaiNameTagger

ner = ThaiNameTagger()
ner.get_ner("24 มิ.ย. 2563 ทดสอบระบบเวลา 6:00 น. เดินทางจากขนส่งกรุงเทพใกล้ถนนกำแพงเพชร ไปจังหวัดกำแพงเพชร ตั๋วราคา 297 บาท")

Corpus: thainer-1.4
- Downloading: thainer-1.4 1.4


  0%|          | 0/1872468 [00:00<?, ?it/s]

[('24', 'NUM', 'B-DATE'),
 (' ', 'PUNCT', 'I-DATE'),
 ('มิ.ย.', 'NOUN', 'I-DATE'),
 (' ', 'PUNCT', 'O'),
 ('2563', 'NUM', 'O'),
 (' ', 'PUNCT', 'O'),
 ('ทดสอบระบบ', 'PART', 'O'),
 ('เวลา', 'NOUN', 'O'),
 (' ', 'PUNCT', 'O'),
 ('6:00', 'NUM', 'B-TIME'),
 (' ', 'PUNCT', 'I-TIME'),
 ('น.', 'NOUN', 'I-TIME'),
 (' ', 'PUNCT', 'O'),
 ('เดินทาง', 'VERB', 'O'),
 ('จาก', 'ADP', 'O'),
 ('ขนส่ง', 'NOUN', 'B-ORGANIZATION'),
 ('กรุงเทพ', 'NOUN', 'I-ORGANIZATION'),
 ('ใกล้', 'ADJ', 'O'),
 ('ถนน', 'NOUN', 'B-LOCATION'),
 ('กำแพงเพชร', 'NOUN', 'I-LOCATION'),
 (' ', 'PUNCT', 'O'),
 ('ไป', 'AUX', 'O'),
 ('จังหวัด', 'VERB', 'B-LOCATION'),
 ('กำแพงเพชร', 'NOUN', 'I-LOCATION'),
 (' ', 'PUNCT', 'O'),
 ('ตั๋ว', 'NOUN', 'O'),
 ('ราคา', 'NOUN', 'O'),
 (' ', 'PUNCT', 'O'),
 ('297', 'NUM', 'B-MONEY'),
 (' ', 'PUNCT', 'I-MONEY'),
 ('บาท', 'NOUN', 'I-MONEY')]

## Word Vector

In [ ]:
import pythainlp.word_vector

pythainlp.word_vector.similarity("คน", "มนุษย์")

In [ ]:
pythainlp.word_vector.doesnt_match(["คน", "มนุษย์", "บุคคล", "เจ้าหน้าที่", "ไก่"])

## Number Spell Out

In [ ]:
from pythainlp.util import bahttext

bahttext(1234567890123.45)

`bahttext()` will round the satang part

In [ ]:
bahttext(1.909)